### Some prerequisite

In [1]:
# set random seed
import random
random.seed(10)


### Setting Hyperparameters

In [2]:
# hyperparameters:
LB = -5
UB = 5
UNIT = 1
NO_UPDATE_TIMES = 10e4
DECREASE_RATE = 0.95

# Define iteration numbers
n = 2

# Set temperature reduction factor
c = 0.5

### Define Objective Function

In [3]:
import numpy as np

# Define the objective function
def f(x_1, x_2):
    return 6*(x_1**2)-6*x_1*x_2+2*(x_2**2)-x_1-2*x_2

print('objective function declaration succeed')

objective function declaration succeed


---
### Define method "get_random": In order to sample form uniform distribution

In [4]:
def get_random(low, high, unit):
    return np.random.uniform(low, high, unit)

---
### Generating Initial Temperature, Initial points

In [5]:
# Initialization
HC = []; RW = []; SA = [];

# Define Initial Temperature
## randomly generate four points,
## and set initial temperature as their mean of objective function values.

import numpy as np
rand_4pts = []
selected_4pts = [[3, 2], [8, 4], [-6, 5], [-1,7]]
for j in range(4):

    rand_4pts.append(f(selected_4pts[j][0],selected_4pts[j][1]))

init_temp = np.mean(rand_4pts)
print(f'The mean objective value of four sample pts is: {init_temp}')

# select an initial design point
# x_1 = get_random(LB, UB+0.1 ,1)
# x_2 = get_random(LB, UB+0.1 ,1)
x_1 = -4
x_2 = 4

HC.append(float(f(x_1, x_2)))
RW.append(float(f(x_1, x_2)))
SA.append(float(f(x_1, x_2)))

SA_X = []; HC_X = []; RW_X = []
SA_X.append([x_1, x_2])
HC_X.append([x_1, x_2])
RW_X.append([x_1, x_2])

print(f'randomly select an initial design point: {x_1, x_2}')

The mean objective value of four sample pts is: 200.5
randomly select an initial design point: (-4, 4)


---
### Main body of Hill Climbing process, start iterating
* Select the state in the neighborhoods of current state which best fit the objective function as the new design state.
* In this implementation, the neighborhoods are set as $(x\pm1\times(0.95)^{\text{update times}},
y\pm1\times(0.95)^{\text{update times}})$.

In [6]:
no_update_times = 0
update_times = 0
current_x_1 = x_1
current_x_2 = x_2
value = f(x_1, x_2)

while(no_update_times < NO_UPDATE_TIMES):
    for i in range(3):
        for j in range(3):
            if (f(current_x_1+(i-1)*(DECREASE_RATE**update_times),
                  current_x_2+(j-1))*(DECREASE_RATE**update_times) <value):
                value = f(current_x_1+i-1,current_x_2+j-1)
                new_x_1 = current_x_1+i-1
                new_x_2 = current_x_1+j-1

    if(f(new_x_1, new_x_2) < f(current_x_1, current_x_2)):
        HC_X.append([new_x_1, new_x_2])
        HC.append(float(f(new_x_1, new_x_2)))
        current_x_1 = new_x_1
        current_x_2 = new_x_2
        no_update_times = 0
        update_times += 1
    else:
        no_update_times += 1

---
### Main body of Random Walk process, start iterating

In [7]:
no_update_times = 0
current_x_1 = x_1
current_x_2 = x_2
while(no_update_times < NO_UPDATE_TIMES):
    attempt_x_1 = get_random(LB, UB+0.1 ,1)
    attempt_x_2 = get_random(LB, UB+0.1 ,1)

    if (f(attempt_x_1,attempt_x_2)<f(current_x_1, current_x_2)):
        RW_X.append([attempt_x_1, attempt_x_2])
        RW.append(float(f(attempt_x_1, attempt_x_2)))
        current_x_1 = attempt_x_1
        current_x_2 = attempt_x_2

    else:
        no_update_times += 1

---
### Main body of SA process, start iterating

In [8]:
# Step 2

## Some Initializations
no_update_times = 0
des_x_1 = x_1
des_x_2 = x_2
temp = init_temp
print(f'init_temp = {init_temp}')
iters_sum = 0
while(no_update_times<=NO_UPDATE_TIMES):

    iter = 0

    while((iter<=n)&(no_update_times<=NO_UPDATE_TIMES)):
        # Compute current_state
        current_state = f(des_x_1, des_x_2)

        # randomly select two uniformly distributed r.v.
        u_1 = get_random(0, 1, 1)
        u_2 = get_random(0, 1, 1)

        # set ranges as +- 6
        r_1 = des_x_1+u_1*12-6
        r_2 = des_x_2+u_2*12-6
        attempt_state = f(r_1, r_2)

        # Step 3 and Step 4
        del_f = attempt_state - current_state

        if (del_f)<0:
            r = get_random(0, 1, 1)
            Prob_of_X_2 = np.exp(-del_f/(1*temp))
            #print(f' r = {r} , $P(X_2)$ = {Prob_of_X_2}')
            if Prob_of_X_2>r:
                des_x_1 = r_1
                des_x_2 = r_2
                #print(f'! assign new design (x_1, x_2) = {(des_x_1, des_x_2)}')
                SA_X.append([r_1, r_2])
                SA.append(float(f(r_1,r_2)))
                current_state = attempt_state
                temp = c*temp
                no_update_times = 0
        else:
            no_update_times +=1

        iter += 1
        iters_sum += 1

init_temp = 200.5


---
### Print the objective function values at each step
* In order to check the solution is descending.

In [9]:
print(HC)
print(RW)
print(SA)
print(len(HC))
print(len(RW))
print(len(SA))
print(HC_X)
print(RW_X)
print(SA_X)

[220.0, 27.0, 14.0, 5.0, 0.0, -1.0, -3.0]
[220.0, 89.85413153722763, 57.84908748423991, 27.42009882521409, 12.649816872176569, -1.4954784893852575, -2.049316576534024, -2.9393062152115066, -3.059001469774289, -3.125384000484167, -3.12634237088566, -3.160415063806335, -3.1642010623188073, -3.164576544900487, -3.166224418660926]
[220.0, 84.97664945697908, 24.316539436881705, 5.970568225669258, 2.079234595081875, -2.734514822394919, -2.9546704035596436, -3.0486633340047993, -3.1615300970023332, -3.165171251339399, -3.166321041411348]
7
15
11
[[-4, 4], [-3, -5], [-2, -2], [-1, -1], [0, 0], [1, 1], [1, 2]]
[[-4, 4], [array([2.83388967]), array([-1.94406259])], [array([-3.62606911]), array([-1.46619097])], [array([-0.67194518]), array([2.99601236])], [array([-1.68451374]), array([-0.98865163])], [array([0.40347138]), array([0.67264303])], [array([1.97311421]), array([2.95798878])], [array([1.07192858]), array([1.85803113])], [array([1.44200909]), array([2.87508623])], [array([1.1685246]), ar

### FE plot

In [11]:
import matplotlib.pyplot as plt
plt.stairs(HC, label='Hill Climbing', baseline=None)
plt.stairs(RW, label='Random Walk', baseline=None)
plt.stairs(SA, label='Simulated Annealing', baseline=None)

plt.xlabel('Iterations')
plt.ylabel('Obejective Function Value')
plt.title('FE plot of the comparison of three algorithms')
# plt.xlim()
# plt.ylim()
plt.grid(axis='y', color='0.95')

plt.legend(loc=1)

plt.show()

AttributeError: module 'matplotlib.pyplot' has no attribute 'stairs'

In [ ]:
import json

with open('result\First_Part\RW.json', 'w') as f:
    json.dump(RW, f)

with open('result\First_Part\HC.json', 'w') as f:
    json.dump(HC, f)

with open('result\First_Part\SA.json', 'w') as f:
    json.dump(SA, f)